In [1]:
import pickle
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam

import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import pickle

DIRECTORY = r'D:\Desktop\Cat_model\dataset\train'
CATEGORIES = ["Earmites","Ringworm"]

IMG_SIZE = 130

data = []

for category in CATEGORIES:
    folder = os.path.join(DIRECTORY, category)
    label = CATEGORIES.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
        data.append([img_arr, label])
random.shuffle(data)


X=[]
y=[]

for features, labels in data:
    X.append(features)
    y.append(labels)


X = np.array(X)
y = np.array(y)

# y = to_categorical(y)
#create a linear stack of layers
model = Sequential()

# Add a convolutional layer with 32 filters, a 3x3 kernel, and ReLU activation
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(130, 130, 3)))

# Add a max pooling layer with a 2x2 pool size to reduce spatial dimensionality of the output
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add a flatten layer to convert the 2D feature maps to a 1D feature vector 
# that can be processed by the fully connected layers
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Add a dense layer with 128 neurons and ReLU activation
model.add(Dense(128, activation='relu'))
#to prevent overfitting
model.add(Dropout(0.5))

# Add an output layer with 2 neurons and a softmax activation
model.add(Dense(2, activation='softmax'))

#RMSprop optimization algorithm for deep learning models
opt = keras.optimizers.RMSprop(learning_rate=0.0001)
#used for multi-class classification problems.
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

base_dir = 'dataset'

train_datagen = image_dataset_from_directory(base_dir+"/train",
                                                  image_size=(130,130),
                                                  subset='training',
                                                  seed = 1,
                                                 validation_split=0.1,
                                                  batch_size= 10,
                                                  label_mode='categorical')
test_datagen = image_dataset_from_directory(base_dir+"/test",
                                                  image_size=(130,130),
                                                  subset='validation',
                                                  seed = 1,
                                                 validation_split=0.1,
                                                  batch_size= 10,
                                                  label_mode='categorical')

model.fit(train_datagen,epochs=25,validation_data=test_datagen,validation_steps=35/32)
model.save('cat.model(newOptimiser2)')

Found 468 files belonging to 2 classes.
Using 422 files for training.
Found 112 files belonging to 2 classes.
Using 11 files for validation.
Epoch 1/25
43/43 [==============================] - 14s 293ms/step - loss: 5.5839 - accuracy: 0.5261 - val_loss: 0.6589 - val_accuracy: 0.6364
Epoch 2/25
43/43 [==============================] - 12s 285ms/step - loss: 1.2310 - accuracy: 0.5355 - val_loss: 0.6896 - val_accuracy: 0.7273
Epoch 3/25
43/43 [==============================] - 12s 268ms/step - loss: 0.9057 - accuracy: 0.5948 - val_loss: 0.5152 - val_accuracy: 0.8182
Epoch 4/25
43/43 [==============================] - 12s 286ms/step - loss: 0.6824 - accuracy: 0.6967 - val_loss: 0.5751 - val_accuracy: 0.7273
Epoch 5/25
43/43 [==============================] - 12s 272ms/step - loss: 0.5706 - accuracy: 0.7417 - val_loss: 0.4926 - val_accuracy: 0.7273
Epoch 6/25
43/43 [==============================] - 12s 269ms/step - loss: 0.5141 - accuracy: 0.7725 - val_loss: 0.8199 - val_accuracy: 0.7273
E

AttributeError: module 'tensorflow' has no attribute 'converters'

In [17]:
from keras.models import load_model
import cv2
import numpy as np

# Load the saved model
model = load_model('cat.model(newOptimiser2)')

# Load the image you want to classify
img = cv2.imread(r'dataset\test\Ringworm\Ringworm-cat-Handsome-5680LF_1_jpg.rf.8f5acea6a261fc1720135f31c6f960cc.jpg')
img = cv2.resize(img, (130, 130))
img = np.reshape(img, [1, 130, 130, 3])

prediction = model.predict(img)

print(prediction)
if prediction[0][0] > prediction[0][1]:
    print("Earmites")
else:
    print("Ringworm")

[[1.5601117e-05 9.9998438e-01]]
Ringworm


In [20]:
# Load test data
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.models import load_model
base_dir = 'dataset'
model = load_model('cat.model(newOptimiser2)')
test_data = image_dataset_from_directory(base_dir+"/test",
                                        #   image_size=(130,130),
                                          subset='validation',
                                          seed = 1,
                                          validation_split=0.9,
                                          batch_size= 10,
                                          label_mode='categorical'
                )

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_data)
print("Test accuracy:", accuracy)

Found 112 files belonging to 2 classes.
Using 100 files for validation.
10/10 [==============================] - 1s 59ms/step - loss: 2.0770 - accuracy: 0.7700
Test accuracy: 0.7699999809265137
